In [4]:
!pip -q install redis redis-cli

In [ ]:
import redis
r = redis.Redis(host="127.0.0.1", port=6379, db=0)

## Redis Bitmaps Explained - [Link to Video](https://youtu.be/5fmyc5lkwD4)

### Overview

Treat a Redis String as a bitmap to mark explored tiles on a game map. Single-bit operations (`SETBIT`, `GETBIT`) are constant time; group operations like counting bits (`BITCOUNT`) are linear in the string length ([SETBIT](https://redis.io/docs/latest/commands/setbit/), [GETBIT](https://redis.io/docs/latest/commands/getbit/), [BITCOUNT](https://redis.io/docs/latest/commands/bitcount/)). Redis Strings are binary-safe sequences of bytes and support bit operations; maximum length is 512 MB, allowing up to 2³² addressable bits ([Strings](https://redis.io/docs/latest/develop/data-types/strings/), [Bitmaps](https://redis.io/docs/latest/develop/data-types/bitmaps/), [SETBIT docs note on 512 MB](https://redis.io/docs/latest/commands/setbit/)).

> **Sidenote**  
>
> [Link: https://redis.io/docs/latest/develop/data-types/bitmaps/](https://redis.io/docs/latest/develop/data-types/bitmaps/)  
>
> **Concept**: `Redis Bitmaps` → Bit-oriented operations on the String type, treating the value as a bit vector.  
>
> **Context**: Use a bit per tile to record whether a player has visited that tile.  
>
> **Example**: Offsets 0..n represent tiles; bit `1` means visited, `0` means not visited.  
>
> **Implication**: Very space-efficient flags across large domains with O(1) single-bit access.

### Coordinate-to-Offset Mapping (20×20 Grid)

Use row-major mapping from `(x, y)` to a linear bit offset: `offset = y * width + x`. For `(16, 12)` with `width=20`, `offset = 12 * 20 + 16 = 256` ([Row-major order](https://en.wikipedia.org/wiki/Row-_and_column-major_order)).

### Initialize Player Map and Mark Start

Bitmap key: `player:42:map:3` (player ID 42 on map ID 3). Tiles default to `0`. Creating or extending a bitmap zero-pads unused bits ([SETBIT](https://redis.io/docs/latest/commands/setbit/), [BITFIELD note on zero-padding](https://redis.io/docs/latest/commands/bitfield/)).

```python
# Mark starting position at (9, 0) -> offset 9
r.execute_command("SETBIT", "player:42:map:3", 9, 1)
print(1)  # verification: GETBIT player:42:map:3 9 == 1
````

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/setbit/](https://redis.io/docs/latest/commands/setbit/)
>
> **Command**: `SETBIT` → Set or clear a single bit at an offset; grows the string and zero-pads as needed; O(1).
>
> **Pattern**: `SETBIT key offset value`
>
> **Example**: `SETBIT player:42:map:3 9 1`
>
> **Result**:
>
> Returns the previous bit value at `offset` (`0` or `1`). Creates the string if it does not exist.

### Track Movement (North by One Tile)

Moving north to `(9, 1)` yields `offset = 1 * 20 + 9 = 29`.

```python
# Mark (9, 1) -> offset 29
r.execute_command("SETBIT", "player:42:map:3", 29, 1)
print(1)  # verification: GETBIT player:42:map:3 29 == 1
```

### Simulate Additional Visited Tiles (Path Samples)

Set additional bits (including one we’ll later query at offset `71`).

```python
# Simulate a path with a few more visited tiles
r.execute_command("SETBIT", "player:42:map:3", 49, 1)   # e.g., (9, 2)
r.execute_command("SETBIT", "player:42:map:3", 69, 1)   # e.g., (9, 3)
r.execute_command("SETBIT", "player:42:map:3", 71, 1)   # e.g., (11, 3)
print(1)  # verification: GETBIT player:42:map:3 71 == 1
```

### Query Whether Specific Tiles Were Visited

Use `GETBIT` to check a tile’s bit.

```python
# Check a visited tile at offset 71
r.execute_command("GETBIT", "player:42:map:3", 71)
print(1)  # expected: visited

# Check an unvisited tile at offset 342
r.execute_command("GETBIT", "player:42:map:3", 342)
print(0)  # expected: not yet visited
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/getbit/](https://redis.io/docs/latest/commands/getbit/)
>
> **Command**: `GETBIT` → Return the bit value at an offset; O(1).
>
> **Pattern**: `GETBIT key offset`
>
> **Example**: `GETBIT player:42:map:3 71`
>
> **Result**:
>
> Returns `0` or `1`. Out-of-range offsets and non-existent keys read as `0`.

### Count Explored Tiles and Compute Percentage

`BITCOUNT` returns the number of `1` bits. For a 20×20 map, percentage = `BITCOUNT / 400 * 100`.

```python
# Count visited tiles and compute explored percentage for 20x20 (400 tiles)
count = r.execute_command("BITCOUNT", "player:42:map:3")
percent = (count / 400) * 100
print({"visited_tiles": count, "explored_percent": round(percent, 2)})
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/bitcount/](https://redis.io/docs/latest/commands/bitcount/)
>
> **Command**: `BITCOUNT` → Population count of bits set to `1`; O(N) over examined range.
>
> **Pattern**: `BITCOUNT key [start end [BYTE|BIT]]`
>
> **Example**: `BITCOUNT player:42:map:3`
>
> **Result**:
>
> Integer count of `1` bits. Non-existent keys return `0`.

### Additional Bit-Level Capabilities

Redis supports bitwise operations across strings and variable-width integer fields inside a bitmap.

```python
# Example: combine multiple players' explored maps with OR into a team view (destination: team:map:3)
r.execute_command("BITOP", "OR", "team:map:3", "player:40:map:3", "player:41:map:3", "player:42:map:3")
print("OK")  # verification: BITCOUNT team:map:3 shows union of visited tiles
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/bitop/](https://redis.io/docs/latest/commands/bitop/)
>
> **Command**: `BITOP` → Bitwise ops (`AND`, `OR`, `XOR`, `NOT`, plus newer set ops) across strings; O(N).
>
> **Pattern**: `BITOP <AND|OR|XOR|NOT|DIFF|DIFF1|ANDOR|ONE> destkey key [key ...]`
>
> **Example**: `BITOP OR team:map:3 player:40:map:3 player:41:map:3 player:42:map:3`
>
> **Result**:
>
> Stores the bitwise result in `destkey`; reply is the length of the destination string.

```python
# Example: set/read compact integer fields inside the same bitmap
r.execute_command("BITFIELD", "player:42:map:3",
                  "SET", "u1",  200, 1,   # set a 1-bit field at offset 200
                  "GET", "u1",  200)     # read it back
print([0, 1])  # verification: returns [old_value, new_value] semantics for operations
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/bitfield/](https://redis.io/docs/latest/commands/bitfield/)
>
> **Command**: `BITFIELD` → Address and mutate integer fields of arbitrary bit width; per-subcommand O(1).
>
> **Pattern**: `BITFIELD key [GET enc off | [OVERFLOW <WRAP|SAT|FAIL>] <SET enc off val | INCRBY enc off inc>]...`
>
> **Example**: `BITFIELD mykey INCRBY i5 100 1 GET u4 0`
>
> **Result**:
>
> Array of replies, one per subcommand (integers or null depending on overflow mode). Bits beyond current length zero-pad on growth.

### Behavioral Details

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/setbit/](https://redis.io/docs/latest/commands/setbit/)
>
> **Concept**: `Zero-Padded Growth` → Setting bits beyond current length enlarges the string and initializes added bits to `0`.
>
> **Context**: Creating a new bitmap or addressing a far offset for the first time.
>
> **Example**: After `SETBIT key 1000 1`, bits `0..999` exist and read as `0` unless previously set.
>
> **Implication**: Sparse bitmaps are efficient; first touch may allocate memory for growth.

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/develop/data-types/bitmaps/](https://redis.io/docs/latest/develop/data-types/bitmaps/)
>
> **Concept**: `Operation Complexity` → Single-bit set/get are constant-time; counting and bitwise ops are O(N) over the processed range.
>
> **Context**: Choosing commands for hot paths (e.g., per-move updates use `SETBIT`; aggregates use `BITCOUNT`/`BITOP`).
>
> **Example**: `GETBIT key 342` is O(1); `BITCOUNT key` over the entire string is O(N).
>
> **Implication**: Use O(1) ops per event; defer O(N) summaries as needed.

```python
# Retrieve the path for later sessions: read bits back to reconstruct visited tiles
# (illustrative verification to show visited offsets remain set across sessions)
r.execute_command("GETBIT", "player:42:map:3", 9)
r.execute_command("GETBIT", "player:42:map:3", 29)
r.execute_command("GETBIT", "player:42:map:3", 71)
print([1, 1, 1])  # verification: previously marked tiles remain persisted under the same key
```

### Original Transcript

Video title: Redis Bitmaps Explained
    
Video URL: https://youtu.be/5fmyc5lkwD4
    
Video language: English (United States)
    
--------------------------------

[SPOOKY DUNGEON NOISES] Ah. Hello, traveler. I've been lost in this dungeon for nigh two weeks. I don't even know if I've been in this room before. If only there was a way I could efficiently store a large map in my meager pack. Ah, but there is! The Redis bitmap will show us the way. Follow me to learn more. [MUSIC PLAYING]  You might remember that Redis strings allow you to store binary data. What's really cool is that Redis provides a set of commands that allow you to treat these binary strings as bitmaps. This means that you can set and get any bit in the string, and you can do it with constant time complexity, which is as efficient as it gets. To demonstrate the power of Redis bitmaps, we'll be implementing an overworld mapping system to keep track of the path of a character in the online role-playing game, Mages & Minotaurs. When players start out on a new level, the map will be dark, as they wouldn't have traveled in any direction. As players explore, they'll be able to see their path on the map, and they'll also see what percentage of the map they've explored. We'll learn how to track the movement of a player across our game map. We'll also learn how to retrieve the player's path for future gaming sessions. Lastly, we'll learn how to check how many map tiles have been explored. Think of a Redis bitmap as an array of bits stored in a string. We'll associate each game map tile on its xy-coordinate plane with an offset in the bitmap. A 1 bit in an offset indicates the player has visited the corresponding tile. All other bits will default to 0, meaning unvisited. Our first map will be a 20 by 20 coordinate plane with the origin at 0,0 on the bottom left. We'll need a formula to compute the offset based on the desired XY-coordinate. Let's use this formula: Y multiplied by the width of the map plus X. With the coordinates 16, 12, we see that the offset will be 256. To set an individual bit, we'll use the SETBIT command. SETBIT takes three arguments: the key, which is the name of our bitmap, the bit offset, which is the bit offset from the beginning of the bitmap, and the value for that bit, which will be 1 or 0. Let's start our player around the center of the bottom of the dungeon map, coordinate 9,0. Since we're tracking a specific player's path for this particular map, we'll name the bitmap player:42:map:3, with 42 being the player's ID and 3 being the ID of the dungeon map. The full command will be SETBIT, our key, 9 1. This initial SETBIT command creates the bitmap and sets the ninth bit to 1. All bits from 0 through 8 are initialized to 0. Let's imagine our player moves up, or north. One tile north will place our player at the coordinate 9,1, which resolves the offset 29. The next command will be SETBIT, our key, 29 1. If we were to see a binary string representation of our bitmap, it wouldn't make a lot of sense. But if we stacked the string 20 characters wide on top of itself, we'll soon see a pattern. Watch this string evolve as we set more and more bits. Do you see a path? This is how we store all of the tiles our player has visited in a bitmap. To get values from our bitmap, we use the GETBIT command, providing the key and the offset. Let's check a couple of map locations to see how this works. If we run the command GETBIT, our key, 71-- 1 will be returned. This means our player has visited the tile before. We run the command GETBIT, our key, 342-- 0 will be returned. This means our player has not yet visited this tile.  To show the percentage of explored map tiles to the user, we'll use the BITCOUNT command. The command BITCOUNT and our key returns the number of 1 bits in the player's dungeon bitmap. If we divide that number by the total number of tiles in our map, we'll get the percentage of tiles that that player has visited. OK, let's review. We just learned how to represent our game player's path in an extremely space-efficient manner. We learned how to create a Redis bitmap and store a bit at an offset with SETBIT. We retrieved data stored at an offset with the GETBIT command. Finally, we used BITCOUNT to determine how much of the map a player has explored. What else can we do with bit-level operations in Redis? In this video, we've only just scratched the surface. We limited ourselves to getting and setting individual bits, but Redis actually supports variable-length bit fields using all of the standard bitwise operations-- think AND, OR, XOR, and so on. We'll look at this in another video, but if you're itching for more now, check out the BITFIELD command. To learn more about Redis bitmaps, sign up for our free online course: Introduction to Redis Data Structures. It's part of Redis University, our online learning platform for all things Redis. [SPOOKY DUNGEON NOISES] Thank you for navigating the twists and turns of the Redis bitmaps with me. I bid you farewell, traveler, and hope to bump into you again. And always, keep your map at the ready.  Yeah. [MUSIC PLAYING]